# BIAFlows

## Nuclei Tracking 2D+t

### Fiji-workflow

The workflow treats the time-dimension as z-dimension and does a 3D-segmentation of the objects. The resulting object slices are then reduced to a center point.

#### Initialize the cytomine client and job

The cytomine client needs some parameters to connect to the server. The host on which the server is running, the public and private keys to make a connection, the id of the project and the id of the software.
The remaining 3 parameters will be passed on to the image analysis workflow.

In [1]:
from getpass import getpass 

publicKey = getpass("Please enter the public key: ")
privateKey = getpass("Please enter the private key: ")

argv = ['--cytomine_public_key', publicKey, 
 '--cytomine_host',  'neubias.cytomine.be',
 '--cytomine_private_key',  privateKey,
 '--cytomine_id_project',  '1695226', 
 '--cytomine_id_software', '3804897',
 '--ij_gauss_radius',  '3',
 '--ij_threshold',  '60',
 '--ij_open_radius', '7']

Please enter the public key: ········
Please enter the private key: ········


Import CytomineJob and Job and update the status information. Set the problem class to particle-tracking (PrtTrk).

In [2]:
import sys
from subprocess import call

from cytomine.models import Job

from neubiaswg5 import CLASS_PRTTRK
from neubiaswg5.helpers import NeubiasJob, prepare_data, upload_data, upload_metrics

jobID=-666
with NeubiasJob.from_cli(argv) as nj:
    nj.job.update(status=Job.RUNNING, progress=0, statusComment="Initialisation...")
    jobID = nj.job.id
    
problem_cls = CLASS_PRTTRK

[2019-01-23 17:39:45,632][INFO] [GET] [currentuser] CURRENT USER - 5739 : neubias | 200 OK
[2019-01-23 17:39:45,687][INFO] [GET] [project] 1695226 : NEUBIAS-BENCHMARK-NUCLEI-TRACKING-NODIVISION | 200 OK
[2019-01-23 17:39:45,740][INFO] [GET] [software] 3804897 : NucleiTracking-ImageJ | 200 OK
[2019-01-23 17:39:45,789][INFO] [GET] [softwareparameter collection] 8 objects | 200 OK
[2019-01-23 17:39:46,243][INFO] [POST] [job] 5850435 : None | 200 OK
[2019-01-23 17:39:46,302][INFO] [GET] [user] 5850441 : JOB[neubias ], 2019-01-23 16:39:45:949 | 200 OK
[2019-01-23 17:39:46,362][INFO] [GET] [currentuser] CURRENT USER - 5850441 : JOB[neubias ], 2019-01-23 16:39:45:949 | 200 OK
[2019-01-23 17:39:46,364][INFO] Job (id:5850435) status update: "None" (status: RUNNING, progress: 0%)
[2019-01-23 17:39:46,457][INFO] [PUT] [job] 5850435 : None | 200 OK
[2019-01-23 17:39:46,531][INFO] [POST] [jobparameter] 5850451 : ij_gauss_radius | 200 OK
[2019-01-23 17:39:46,606][INFO] [POST] [jobparameter] 5850457 

#### Create local directories and download images

Create the local working directories in a subfolder jobID of the user's home folder.

- in: input images
- out: output images
- ground_truth: ground truth images
- tmp: temporary path

In [4]:
in_images, gt_images, in_path, gt_path, out_path, tmp_path = prepare_data(problem_cls, nj, is_2d=False, **nj.flags)

[2019-01-23 17:40:00,948][INFO] Job (id:5850435) status update: "Downloading images (to /home/baecker/5850435/in)..." (status: SUCCESS/TERMINATED, progress: 25%)
[2019-01-23 17:40:01,113][INFO] [PUT] [job] 5850435 : None | 200 OK
[2019-01-23 17:40:01,176][INFO] [GET] [imagegroup collection] 2 objects | 200 OK
[2019-01-23 17:40:01,645][INFO] File downloaded successfully from http://neubias.cytomine.be/api/imagegroup/4042304/download
[2019-01-23 17:40:02,031][INFO] File downloaded successfully from http://neubias.cytomine.be/api/imagegroup/4042708/download


#### Call the image analysis workflow 

In [5]:
from subprocess import call

nj.job.update(progress=25, statusComment="Launching workflow...")
command = "/usr/bin/xvfb-run ./ImageJ-linux64 -macro NucleiTracking.ijm " \
		"\"input={}, output={}, gauss_rad={}, threshold={}, open_rad={}\" -batch".format(in_path, out_path, nj.parameters.ij_gauss_radius, nj.parameters.ij_threshold, nj.parameters.ij_open_radius)

return_code = call(command, shell=True, cwd="./Fiji.app")  # waits for the subprocess to return

if return_code != 0:
    err_desc = "Failed to execute the ImageJ macro (return code: {})".format(return_code)
    nj.job.update(progress=50, statusComment=err_desc)
    raise ValueError(err_desc)
nj.job.update(progress=30, statusComment="Workflow finished...")


[2019-01-23 17:40:04,640][INFO] Job (id:5850435) status update: "Launching workflow..." (status: SUCCESS/TERMINATED, progress: 1%)
[2019-01-23 17:40:04,787][INFO] [PUT] [job] 5850435 : None | 200 OK
[2019-01-23 17:40:12,500][INFO] Job (id:5850435) status update: "Workflow finished..." (status: SUCCESS/TERMINATED, progress: 25%)
[2019-01-23 17:40:12,661][INFO] [PUT] [job] 5850435 : None | 200 OK


#### Create and upload annotations

In [6]:
from neubiaswg5.exporter.mask_to_points import mask_to_points_3d
import io
import os
from skimage import io
from cytomine.models import Annotation,AnnotationCollection, ImageGroupCollection, ImageInstance, ImageSequenceCollection
from itertools import groupby
from shapely.geometry import LineString, Point, MultiPoint
from shapely.affinity import affine_transform
import numpy as np
   
for image in nj.monitor(in_images, start=60, end=80, period=0.1, prefix="Extracting and uploading tracks from point-masks"):
    file = "{}.tif".format(image.id)
    path = os.path.join(out_path, file)
    data = io.imread(path)

    # extract objects
    objects = mask_to_points_3d(data, time=True)

    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.time)    
    objects = sorted(objects, key=lambda annotationSlice: annotationSlice.label)    
    grouped = groupby(objects,  key=lambda annotationSlice: annotationSlice.label)
    
    image_sequences = ImageSequenceCollection().fetch_with_filter("imagegroup", image.id)
    
    depth_to_image = {iseq.time: iseq.image for iseq in image_sequences}
    height = ImageInstance().fetch(image_sequences[0].image).height
    
    collection = AnnotationCollection()
    
    groups = []
    uniquekeys = []
    for k, g in grouped:
        groups.append(list(g))      # Store group iterator as a list
        uniquekeys.append(k)
    
    for key, group in zip(uniquekeys, groups):
        listOfPoints = []
        for oSlice in group:
            listOfPoints.append((oSlice.polygon.x, oSlice.polygon.y))
        # shape = LineString(listOfPoints)
        shape = MultiPoint(listOfPoints)
        for oSlice2 in group:
            polygon = affine_transform(shape, [1, 0, 0, -1, 0, height])
            annotation = Annotation(location=polygon.wkt, id_image=depth_to_image[oSlice2.time], id_project=nj.parameters.cytomine_id_project, property=[{"key": "index", "value": str(oSlice2.label)}])
            collection.append(annotation)
        
    collection.save()
    print("Found {} objects in this image {}.".format(len(objects), image.id))
    
    

[2019-01-23 17:40:20,291][INFO] Job (id:5850435) status update: "Extracting and uploading tracks from point-masks (1/1)." (status: RUNNING, progress: 30%)
[2019-01-23 17:40:24,250][INFO] [PUT] [job] 5850435 : None | 200 OK
[2019-01-23 17:40:24,402][INFO] [GET] [imagesequence collection] 10 objects | 200 OK
[2019-01-23 17:40:24,460][INFO] [GET] [imageinstance] 4041830 : /1539945624298/ShortMovie.ome_Z0_C0_T5.tiff | 200 OK
[2019-01-23 17:40:31,404][INFO] [POST] algoannotations 5850549,5850563,5850577,5850591,5850605,5850619,5850633,5850647,5850661,5850675,5850689,5850702,5850715,5850730,5850743,5850756,5850769,5850782,5850795,5850808,5850821,5850834,5850847,5850860,5850873,5850886,5850899,5850912,5850925,5850938,5850951,5850964,5850977,5850990,5851003,5851016,5851029,5851042,5851055,5851068,5851081,5851094,5851107,5851120,5851133,5851146,5851159,5851172,5851185,5851198,5851211,5851224,5851237,5851250,5851263,5851276,5851289,5851302,5851315,5851328,5851341,5851354,5851367,5851380,5851393,

[2019-01-23 17:41:28,551][INFO] [POST] algoannotations 5862325,5862338,5862351,5862364,5862377,5862390,5862403,5862416,5862429,5862442,5862455,5862468,5862481,5862494,5862507,5862520,5862533,5862546,5862559,5862572,5862585,5862598,5862611,5862624,5862637,5862650,5862663,5862676,5862689,5862702,5862715,5862728,5862741,5862754,5862767,5862780,5862793,5862806,5862819,5862832,5862845,5862858,5862871,5862888,5862901,5862914,5862927,5862940,5862953,5862966,5862979,5862992,5863005,5863018,5863031,5863044,5863057,5863070,5863083,5863096,5863111,5863124,5863137,5863150,5863163,5863176,5863189,5863202,5863215,5863228,5863241,5863254,5863267,5863280,5863293,5863306,5863319,5863332,5863345,5863358,5863371,5863384,5863397,5863410,5863423,5863436,5863449,5863462,5863475,5863488,5863501,5863514,5863527,5863540,5863553,5863566,5863579,5863592,5863605,5863618 added | 200 OK
[2019-01-23 17:41:35,738][INFO] [POST] algoannotations 5863631,5863644,5863657,5863670,5863683,5863696,5863709,5863722,5863735,586

#### Calculate and upload metrics

In [7]:
# 5. Compute and upload the metrics
nj.job.update(progress=80, statusComment="Computing and uploading metrics (if necessary)...")
upload_metrics(problem_cls, nj, in_images, gt_path, out_path, tmp_path, **nj.flags)

nj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")

[2019-01-23 17:42:34,535][INFO] Job (id:5850435) status update: "Computing and uploading metrics (if necessary)..." (status: RUNNING, progress: 60%)
[2019-01-23 17:42:34,721][INFO] [PUT] [job] 5850435 : None | 200 OK


FileNotFoundError: [Errno 2] No such file or directory: '/home/baecker/5850435/tmp/intracks.xml.score.txt'

#### Cleanup - remove the files and folders that have been downloaded or created.

In [ ]:
for image in in_images:
    file = str(image.id)
    path = out_path + "/" + file + ".tif"
    os.remove(path);
    path = in_path + "/" + file  + ".tif"
    os.remove(path);
    path = gt_path + "/" + file  + ".tif"
    os.remove(path);
    
tmpData = ['intracks.xml', 'intracks.xml.score.txt', 'reftracks.xml']
for file in tmpData:
    path = tmp_path + "/" + file
    os.remove(path);
if os.path.exists(working_path):
    os.rmdir(in_path)
    os.rmdir(out_path)
    os.rmdir(gt_path)
    os.rmdir(tmp_path)
    os.rmdir(working_path)

#### End the job and the connection to cytomine

In [ ]:
nj.job.update(status=Job.TERMINATED, progress=100, statusComment="Finished.")
print("END.")